<a href="https://colab.research.google.com/github/filmfer/2020_FluxSat_MDPI_RemoteSensing/blob/master/docs/examples/satellite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segment Anything Model for Geospatial Data 

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/satellite.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

This notebook shows how to use segment satellite imagery using the Segment Anything Model (SAM) with a few lines of code. 

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

## Install dependencies

Uncomment and run the following cell to install the required dependencies.


In [ ]:
# %pip install segment-geospatial leafmap localtileserver

## Import libraries

In [ ]:
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap('SATELLITE')
m

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [ ]:
image = 'satellite.tif'
# image = '/path/to/your/own/image.tif'

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite')

In [ ]:
m.add_raster(image, layer_name='Image')
m

![](https://i.imgur.com/KAm84IY.png)

## Initialize SAM class

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

## Segment the image

In [ ]:
mask = 'segment.tiff'
sam.generate(image, mask)

## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [ ]:
vector = 'segment.gpkg'
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [ ]:
shapefile = 'segment.shp'
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [ ]:
style = {
    'color': '#3388ff',
    'weight': 2,
    'fillColor': '#7c4185',
    'fillOpacity': 0.5,
}
m.add_vector(vector, layer_name='Vector', style=style)
m

![](https://i.imgur.com/Ysq3u7E.png)